In [100]:
import pandas as pd
import math
import csv
import random
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [101]:
# 赋予每只队伍基础Elo等级分
base_elo = 1600
team_elos = {}
team_stats = {}
X = []
y = []

In [102]:
# 根据每只队伍M、O、T表格中的数据进行初始化
def initialize_data(Mstat, Ostat, Tstat):
    # 去除无关数据
    new_Mstat = Mstat.drop(['Rk', 'Arena'], axis=1)
    new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
    new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)
    
    '''
    merge函数类似于SQL的join，可以将不同数据集通过某些字段（属性）进行合并
    how:默认为inner，有inner/outer/left/right
    on:根据某个字段连接，必须存在于两个DataFrame中
    
    set_index:可将普通列设置为单索引或复合索引
    将Team设置为team_stats1的索引
    ''' 
    team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
    team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team' )
    return team_stats1.set_index('Team', inplace=False, drop=True)

In [103]:
# 赋予初始Bash_elo

def get_elo(team):
    try:
        return team_elos[team]
    except:
        # 当最初没有elo时，给每个队伍都赋予base_elo
        team_elos[team] = base_elo
        return team_elos[team]

In [104]:
# 定义计算每只球队elo值得函数
def calc_elo(win_team, lose_team):
    winner_rank = get_elo(win_team)
    loser_rank = get_elo(lose_team)
    
    rank_diff = winner_rank - loser_rank
    exp = (rank_diff * -1) / 400
    odds = 1 / (1 + math.pow(10,exp))
    
    # 根据rank级别修改k值
    if winner_rank < 2100:
        k = 32
    elif winner_rank >= 2100 and winner_rank < 2400:
        k = 24
    else:
        k = 16
        
    # 更新rank值
    new_winner_rank = round(winner_rank + (k * (1 - odds)))
    new_loser_rank = round(loser_rank + (k * (0 - odds)))
    return new_winner_rank, new_loser_rank

In [105]:
# 根据每场2018-2019每场比赛得数据，建立数据集
def build_dataSet(all_data):
    print("Buileding data set...")
    X = []
    skip = 0
    for index, row in all_data.iterrows():
        
        Wteam = row['WTeam']
        Lteam = row['LTeam']
        
        # 获取最初得elo值
        team1_elo = get_elo(Wteam)
        team2_elo = get_elo(Lteam)
        
        # 给主场比赛得队伍加上100的elo值
        if row['WLoc'] == 'H':
            team1_elo += 100
        else:
            team2_elo += 100
            
            
        # 将elo当作评价每个队伍的第一个特征值
        team1_features = [team1_elo]
        team2_features = [team2_elo]
        
        
        for key, value in team_stats.loc[Wteam].iteritems():
            team1_features.append(value)
        for key, value in team_stats.loc[Lteam].iteritems():
            team2_features.append(value)
            
            
        if random.random() > 0.5:
            X.append(team1_features + team2_features)
            y.append(0)
        else:
            X.append(team2_features + team1_features)
            y.append(1)
            
        if skip == 0:
            print('X', X)
            skip = 1
            
        # 根据这场比赛的数据更新队伍的elo值
        new_winner_rank, new_loser_rank = calc_elo(Wteam, Lteam)
        team_elos[Wteam] = new_winner_rank
        team_elos[Lteam] = new_loser_rank
    
    # nan_to_num:使用0代替nan
    return np.nan_to_num(X), y

In [106]:
# 进行胜负判读预测
def predict_winner(team_1, team_2, model):
    features = []
    
    # team1:客场队伍
    features.append(get_elo(team_1))
    for key,value in team_stats.loc[team_1].iteritems():
        features.append(value)
        
    # team2:主场队伍
    features.append(get_elo(team_2) + 100)
    for key, value in team_stats.loc[team_2].iteritems():
        features.append(value)
        
    features = np.nan_to_num(features)
    return model.predict_proba([features])
    
    
    

In [107]:
if __name__ == "__main__":
    
    Mstat = pd.read_csv('2018-2019Miscellaneous Stats.csv')
    Ostat = pd.read_csv('2018-2019Opponent Per Game Stats.csv')
    Tstat = pd.read_csv('2018-2019Team Per Game Stats.csv')
    
    team_stats = initialize_data(Mstat, Ostat, Tstat)
    
    result_data = pd.read_csv('2018-2019_result.csv')
    
    X, y = build_dataSet(result_data)
    
    
    # 训练网络模型
    print("Fitting on %d game samples..." % len(X))
    
    model = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
    model.fit(X, y)
    
    # 利用10折交叉验证计算训练正确率
    print("Doing crodd-validation...")
    print(cross_val_score(model, X, y, cv=10, scoring='accuracy', n_jobs=-1).mean())
    
    print("Predicting on new schedule...")
    schedule1920 = pd.read_csv('2019-2020_Schedule.csv')
    result = []
    
    for index, row in schedule1920.iterrows():
        team1 = row['Vteam']
        team2 = row['Hteam']
        pred = predict_winner(team1, team2, model)
        prob = pred[0][0]
        if prob > 0.5:
            winner = team1
            loser = team2
            result.append([winner, loser, prob])
        else:
            winner = team2
            loser = team1
            result.append([winner, loser, 1-prob])
            
    with open('2019-2020_Schedule.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['win', 'lose', 'probability'])
        writer.writerows(result)
        print("Done.")
    

Buileding data set...
X [[1600, 26.4, 51.0, 31.0, 48.0, 34.0, 2.7, -0.44, 2.25, 112.6, 110.0, 2.6, 101.6, 0.312, 0.342, 0.574, 0.532, 12.9, 24.5, 0.24100000000000002, 0.512, 11.1, 78.6, 0.20600000000000002, 838342.0, 20447.0, 41.7, 91.5, 0.455, 10.3, 30.0, 0.342, 31.4, 61.5, 0.511, 18.8, 24.5, 0.768, 10.0, 33.5, 43.5, 23.4, 7.7, 4.1, 12.7, 22.1, 112.5, 41.5, 88.2, 0.47100000000000003, 10.8, 30.2, 0.359, 30.7, 58.0, 0.529, 21.2, 27.5, 0.7709999999999999, 10.9, 36.9, 47.8, 26.9, 7.4, 5.3, 14.9, 21.3, 115.2, 1700, 25.7, 49.0, 33.0, 52.0, 30.0, 4.44, -0.54, 3.9, 112.2, 107.8, 4.4, 99.6, 0.215, 0.381, 0.5670000000000001, 0.534, 11.5, 21.6, 0.17300000000000001, 0.514, 13.4, 77.0, 0.198, 763584.0, 18624.0, 39.5, 88.1, 0.44799999999999995, 11.5, 33.5, 0.344, 28.0, 54.6, 0.513, 17.4, 22.8, 0.764, 10.4, 35.5, 45.9, 23.7, 6.8, 3.9, 15.1, 19.5, 108.0, 42.1, 90.5, 0.465, 12.6, 34.5, 0.365, 29.5, 56.0, 0.527, 15.6, 19.5, 0.802, 9.8, 34.7, 44.5, 26.3, 8.6, 5.3, 12.8, 20.4, 112.4]]
Fitting on 1312 gam